In [63]:
%%writefile diet_plans.txt
Breakfast,Oatmeal with banana,300
Lunch,Grilled chicken salad,500
Dinner,Salmon and veggies,450
Snack,Yogurt with nuts,200
Breakfast,Scrambled eggs and toast,350
Lunch,Tuna sandwich,450
Dinner,Spaghetti with meat sauce,600
Snack,Fruit smoothie,250

Overwriting diet_plans.txt


#Header Class

In [64]:
%%writefile DietPlans.h
#ifndef DIETPLANS_H
#define DIETPLANS_H
#include<iostream>
using namespace std;

class DietPlans{
  private:
    string type_; //Breakfast, Lunch, Dinner, Snack
    string plan_; //Patmeal with banana, Grilled chicken salad, Salmon and veggies, Yogurt with nuts
    int calories_;

  public:
    DietPlans();
    DietPlans(string, string, int);
    string getType() const;
    string getPlan() const;
    int getCalories() const;
    void setType(string);
    void setPlan(string);
    void setCalories(int);
    void setMeal(string, string, int);
};

#endif

Overwriting DietPlans.h


In [65]:
%%writefile DietPlansImp.cpp

#include "DietPlans.h"
#include<iostream>
#include<string>
using namespace std;

DietPlans::DietPlans(){
  type_ = "";
  plan_ = "";
  calories_ = 0;
}

DietPlans::DietPlans(string t, string p, int c){
  type_ = t;
  plan_ = p;
  calories_ = c;
}

string DietPlans::getType() const{
  return type_;
}

string DietPlans::getPlan() const{
  return plan_;
}

int DietPlans::getCalories() const{
  return calories_;
}

void DietPlans::setType(string t){
  type_ = t;
}

void DietPlans::setPlan(string p){
  plan_ = p;
}

void DietPlans::setCalories(int c){
  calories_ = c;
}

void DietPlans::setMeal(string t, string p, int c){
  setType(t);
  setPlan(p);
  setCalories(c);
}

Overwriting DietPlansImp.cpp


In [66]:
%%writefile main.cpp

#include <iostream>
#include <fstream>
#include <iomanip>
#include "DietPlans.h"
using namespace std;

//Constants
const int MIN_F_INTAKE = 1200; //minimum calories intake for female
const int MIN_M_INTAKE = 1500; //minimum calories intake for male
const int NUM_PLANS = 20;      //number of plans

//Function Prototypes
string isRecorded(int);
int selectPlan(const DietPlans*, int, string);
string trackCalories(const DietPlans*, int[], int&);
void calcCalDiff(int, int, char, int&);
void defineSuggestion(const DietPlans*, const int[], int, int);
void suggestMultipleChange(const DietPlans&, const DietPlans&, int&, bool&);
void suggestSingleChange(const DietPlans&, const DietPlans&, int&);
void suggestPlan(const DietPlans*,int*, int, int);
void printPlan(const DietPlans*, int[], int);

//Main Function
int main();

int main(){
  DietPlans* dpArray = new DietPlans[NUM_PLANS];
  ifstream dietFile;
  dietFile.open("diet_plans.txt");

  if(!dietFile){ //Check if dietfile is open
    cout << "Error opening file" << endl;
    return 1;
  }

  string type = ""; //meal type
  string plan = ""; //meal plan(food plan)
  int calories = 0; //calories of meal
  int numPlans = 0;

  while (getline(dietFile, type, ',') && getline(dietFile, plan, ',') && dietFile >> calories) {
    dietFile.ignore(); // to skip the newline character after reading calories
    dpArray[numPlans++].setMeal(type, plan, calories);
  }

  dietFile.close(); //close file

  int choice = 0; //daily diet menu option choice

  //meal plan selection, stores the plan index according dpArray
  //use pointer so suggestPlan() can modify the array within the function
  int *dailyPlan = new int[4];
  for(int i = 0; i < 4; i++){ //initialize pointer array values to -1
    dailyPlan[i] = -1;
  }

  int totalCal = 0; //total calories intake of the current selected meal plans
  int calories_diff = 0;

  //refer previous variables from other classes
  char gender = 'F'; //get the gender of user
  int rec_cal = 1650; //get recommended calories intake from the tdee calculation

  //variables that differ according the gender and rec_cal variables
  int min = (gender == 'F')? MIN_F_INTAKE : MIN_M_INTAKE;
  int cal_limit = rec_cal + 500; //calories intake limit

  do{
    cout << "+--------------------------------+" << endl;
    cout << "|           Daily Diet           |" << endl;
    cout << "+--------------------------------+" << endl;
    cout << left << fixed;
    cout << "|" << setw(15) << " [1] Breakfast" << setw(17) << isRecorded(dailyPlan[0]) << "|"<< endl;
    cout << "|" << setw(15) << " [2] Lunch" << setw(17) << isRecorded(dailyPlan[0]) << "|"<< endl;
    cout << "|" << setw(15) << " [3] Dinner" << setw(17) << isRecorded(dailyPlan[0]) << "|"<< endl;
    cout << "|" << setw(15) << " [4] Snack" << setw(17) << isRecorded(dailyPlan[0]) << "|"<< endl;
    cout << "|" << setw(32) << " [5] Suggested Plan" << "|"<< endl;
    cout << "|" << setw(32) << " [6] Done" << "|"<< endl;
    cout << "+--------------------------------+" << endl;
    cout << "Recommended Calories Intake: " << rec_cal << endl;
    cout << "Current" << trackCalories(dpArray, dailyPlan, totalCal);
    calcCalDiff(totalCal, cal_limit, gender, calories_diff);
    defineSuggestion(dpArray, dailyPlan, calories_diff, numPlans);
    cout << "Enter choice: ";
    cin >> choice;

    switch(choice){
      case 1:
        dailyPlan[0] = selectPlan(dpArray, numPlans, "Breakfast");
        break;
      case 2:
        dailyPlan[1] = selectPlan(dpArray, numPlans, "Lunch");
        break;
      case 3:
        dailyPlan[2] = selectPlan(dpArray, numPlans, "Dinner");
        break;
      case 4:
        dailyPlan[3] = selectPlan(dpArray, numPlans, "Snack");
        break;
      case 5:
        suggestPlan(dpArray, dailyPlan, rec_cal, numPlans);
        break;
      case 6:
        printPlan(dpArray, dailyPlan, totalCal);
        break;
      default:
        cout << "Invalid option" << endl;
    }

  }while(choice != 6);


  delete[] dpArray; //delete dynamic array
  dpArray = nullptr;

  return 0;
}

//Function Declaration

//printPlan()
// prints the final diet plan
void printPlan(const DietPlans dp[], int plan[], int total){
  cout << "+------------------------------------------------------+" << endl;
  cout << "|                      Diet Plan                       |" << endl;
  cout << "+------------------------------------------------------+" << endl;
  cout << left << fixed;
  for(int i = 0; i < 4; i++){
    cout << "| " << setw(10) << dp[plan[i]].getType() << ": " << setw(30) << dp[plan[i]].getPlan()
                 << " (" <<  dp[plan[i]].getCalories() << " cal) |" << endl;
  }
  cout << "| Total Calories Intake: " << setw(30) << total << "|" << endl;
  cout << "+------------------------------------------------------+" << endl;
}

//suggestPlan()
// Suggests a full plan which is near to the recommended calories intake
// The suggested plan will be passed by assigning to the plan[] pointer array
void suggestPlan(const DietPlans dp[], int plan[], int rec, int dpSize) {
    string mealTypes[4] = {"Breakfast", "Lunch", "Dinner", "Snack"}; // Meal types to iterate over
    int temp[4] = {-1, -1, -1, -1}; // Temporary array to store the indices of selected meal plans
    int totalCalories = 0; // Track total calories of the suggested plan
    int limit = rec + 500; // Limit for total calories

    cout << "+-----------------------------------------------------+" << endl;
    cout << "|                   Suggested Plan                    |" << endl;
    cout << "+-----------------------------------------------------+" << endl;
    cout << left << fixed;

    // Loop through each meal type
    for (int t = 0; t < 4; t++) {
        // If recommended intake is met or exceeded, stop suggesting further meals
        if (totalCalories >= rec) break;

        int validMeals[dpSize]; // Array to hold valid plan indices for current meal type
        int validCount = 0;
        // Filter meal plans for the current type that fit within the remaining calorie budget
        for (int i = 0; i < dpSize; i++) {
            if (dp[i].getType() == mealTypes[t]) {
                // Calculate what the total calories would be if this meal was added
                int projectedCalories = totalCalories + dp[i].getCalories();
                // Accept the meal if it doesn’t push total calories over rec + 500
                if (projectedCalories <= limit) {
                    validMeals[validCount++] = i;
                }
            }
        }
        // If there are any valid meals for this type, pick one randomly
        if (validCount > 0) {
            int selectedIndex = validMeals[rand() % validCount]; // Pick one randomly
            temp[t] = selectedIndex; // Store selected plan index
            totalCalories += dp[selectedIndex].getCalories(); // Add its calories to total

            // Print the selected plan in formatted output
            cout << "| " << setw(9) << mealTypes[t] << ": " << setw(30) << dp[selectedIndex].getPlan()
                 << " (" << dp[selectedIndex].getCalories() << " cal) |" << endl;
        }
    }
    // Footer for plan table
    cout << "+-----------------------------------------------------+" << endl;
    // Show total calories of the plan
    cout << "Total Calories: " << totalCalories << endl;

    // Final check: ensure the total is within acceptable range [1200, rec + 500]
    if (totalCalories < 1200 || totalCalories > limit) {
        cout << "Unable to generate a valid plan within required calorie range.\n";
        return;
    }
    // Ask user whether to accept and use the suggested plan
    cout << "Use suggested plan? (Y/N): ";
    char choice;
    cin >> choice;

    // If user accepts, copy selected plan indices to output array
    if (choice == 'Y' || choice == 'y') {
        for (int i = 0; i < 4; i++) {
            plan[i] = temp[i];
        }
        cout << "Plan used successfully.\n";
    } else {
        cout << "Plan not used.\n";
    }
}

//suggestMultipleChange()
// suggests multiple meal plan change to meet limit range
// &suggest: status for meal type change suggestion(if suggested to change = true)
void suggestMultipleChange(const DietPlans &lp, const DietPlans &op, int &diff, bool &suggest){
  //end the process if the looped plan calories is higher than the original plan
  if(op.getCalories() < lp.getCalories())
    return;

  diff -= op.getCalories() - lp.getCalories(); //calories difference of the current looped plan and original plan
  cout << setw(15) << left << "" << "Change " << lp.getType() << " to " << lp.getPlan() << endl;
  suggest = true;
}

//suggestSingleChange()
// only suggest changing 1 meal plan to meet limit range
// & for lp and op, used to refer original object, prevent new object creation
// lp: current looped meal plan
// op: original selected meal plan
void suggestSingleChange(const DietPlans &lp, const DietPlans &op, int &diff){
  int x = op.getCalories() - lp.getCalories(); //calories difference of the current looped plan and original plan
  if(x >= diff){
    cout << setw(15) << left << "" << "Change " << lp.getType() << " to " << lp.getPlan() << endl;
    diff = 0;
  }
}

//defineSuggestion()
// define and prints meal suggestions
void defineSuggestion(const DietPlans dp[], const int plans[], int diff, int dpSize) {
  string mealTypes[4] = {"Breakfast", "Lunch", "Dinner", "Snack"};
  bool selected[4] = {plans[0] != -1, plans[1] != -1, plans[2] != -1, plans[3] != -1};

  if (diff < 0) {
    // Suggest adding a meal if under the minimum intake
    for (int i = 0; i < 4; ++i) {
      if (!selected[i]) {
        for (int j = 0; j < dpSize; ++j) {
          if (dp[j].getType() == mealTypes[i] && dp[j].getCalories() <= -diff) {
            cout << "Suggestion: Add a " << mealTypes[i] << " Meal - " << dp[j].getPlan() << endl;
            return;
          }
        }
      }
    }
  } else if (diff > 0) {
    // Suggest removing snack first if it exists
    if (selected[3]) {
      cout << setw(15) << left << "Suggestion: " << "Don't eat snacks." << endl;
      diff -= dp[plans[3]].getCalories();
    }

    // Try single meal change to fix over-intake
    for (int i = 0; i < 4; ++i) {
      if (!selected[i]) continue;
      for (int j = 0; j < dpSize; ++j) {
        if (dp[j].getType() == mealTypes[i] && j != plans[i]) {
          suggestSingleChange(dp[j], dp[plans[i]], diff);
          if (diff <= 0) return;
        }
      }
    }

    // If still over, suggest multiple changes
    bool suggested[4] = {false, false, false, false};
    for (int i = 0; i < dpSize; ++i) {
      string type = dp[i].getType();
      for (int m = 0; m < 4; ++m) {
        if (type == mealTypes[m] && selected[m] && i != plans[m] && !suggested[m]) {
          suggestMultipleChange(dp[i], dp[plans[m]], diff, suggested[m]);
          if (diff <= 0) return;
        }
      }
    }
  }
}


//calcCalDiff()
// calculates the difference of the total calories with the recomended range
void calcCalDiff(int tot, int limit, char gender, int &diff){
  int min = (gender == 'F')? MIN_F_INTAKE : MIN_M_INTAKE;

  if(tot < min){
    diff = tot - min; //gets a - value, to indicate it doesn't meet the min limit
    cout << "< Minimum calories intake not met >" << endl;
  }
  else if(tot > limit){
    diff = tot - limit; //gets + value, to indicate it exceed
    cout << "< Calories intake exceed recommended limit >" << endl;
  }
  else{
    diff = 0;
  }
}

//trackCalories()
// calculate and return the total calories intake of the selected meal plans
string trackCalories(const DietPlans dp[], int plan[], int &tot){
  tot = 0;
  for(int i = 0; i < 4; i++){
    if(plan[i] != -1){
      tot += dp[plan[i]].getCalories();
    }
  }
  return ("Calories Intake:" + to_string(tot) + "\n");
}

//selectPlan()
// prints all the meal plans for a specific meal type
// then prompts user to select a meal plan
// returns the dpArray index of selected meal plan
int selectPlan(const DietPlans dp[], int plans, string meal){
  int opt = 1;    //plan option index
  int plan = 0;   //plan selection
  int j = 0;      //index for dpIndex
  int dpIndex[plans]; //array of the indexes of the particular meal types

  cout << meal << "Meal Plan: " << endl;

  for(int i = 0; i < plans; i++){
    if(dp[i].getType() == meal){
      cout << "[" << opt++ << "] " << dp[i].getPlan() << " (" << dp[i].getCalories() << " calories)" << endl;
      dpIndex[j++] = i;
    }
  }

  cout << "Enter Plan: ";
  cin >> plan;

  return dpIndex[plan - 1];
}

//isRecorded()
// prints a (Recorded) behind the meal type in the menu to indicate that
// the meal plan for the meal type is already chosen
string isRecorded(int meal){
  if(meal == -1){
    return "";
  }
  else{
    return " (Recorded)";
  }
}

Overwriting main.cpp


In [67]:
%%shell
g++ DietPlans.h DietPlansImp.cpp main.cpp -o HealthJournal.elf
./HealthJournal.elf

+--------------------------------+
|           Daily Diet           |
+--------------------------------+
| [1] Breakfast                  |
| [2] Lunch                      |
| [3] Dinner                     |
| [4] Snack                      |
| [5] Suggested Plan             |
| [6] Done                       |
+--------------------------------+
Recommended Calories Intake: 1650
CurrentCalories Intake:0
< Minimum calories intake not met >
Suggestion: Add a Breakfast Meal - Oatmeal with banana
Enter choice: 5
+-----------------------------------------------------+
|                   Suggested Plan                    |
+-----------------------------------------------------+
| Breakfast: Scrambled eggs and toast       (350 cal) |
| Lunch    : Grilled chicken salad          (500 cal) |
| Dinner   : Spaghetti with meat sauce      (600 cal) |
| Snack    : Fruit smoothie                 (250 cal) |
+-----------------------------------------------------+
Total Calories: 1700
Use suggested p